In [15]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
data = pd.read_csv('/content/정제파일.csv',encoding='utf-8')
column_name='제목_본문'
cv = CountVectorizer(max_features=99999, ngram_range=(1, 1))
tdm = cv.fit_transform(data[column_name])
word_count_tf = pd.DataFrame({'단어': cv.get_feature_names_out(), '빈도': tdm.sum(axis=0).flat})
word_count_tf = word_count_tf.sort_values('빈도', ascending=False)
word_count_tf = word_count_tf.reset_index(drop=True)
word_count_tf.index = word_count_tf.index + 1

In [16]:
word_count_tf.to_csv("빈도.csv",encoding='cp949')

In [17]:
# 상위 빈도 N개 기준
from itertools import combinations

# 상위 30개 단어 추출
top_30_words = word_count_tf['단어'].head(30).tolist()
# 리뷰 데이터에서 자주 등장하는 단어 30개를 리스트로 추출

# 원본 리뷰 데이터 불러오기
data = data.astype('str')
# 데이터의 모든 값을 문자열로 변환

# '[' ']', ',' 그리고 "'"를 삭제하는 코드
chars_to_remove = r"[\[\]',]"
data = data.replace(chars_to_remove, '', regex=True)
# 텍스트 데이터에서 불필요한 문자들을 제거하여 분석 준비

# 동시 출현 빈도 계산을 위한 빈 딕셔너리 생성
co_occurrence = {word: {other_word: 0 for other_word in top_30_words} for word in top_30_words}
# 각 단어 쌍의 동시 출현 빈도를 저장할 빈 딕셔너리 생성

# 각 리뷰에 대해 동시 출현 빈도 계산
for review in data[column_name]:
    # 리뷰에서 상위 30개 단어 찾기
    words_in_review = set(word for word in review.split() if word in top_30_words)
    # 리뷰에서 상위 30개 단어만 골라 세트로 만듦

    # 단어 쌍에 대해 동시 출현 빈도 증가
    for word1, word2 in combinations(words_in_review, 2):
        co_occurrence[word1][word2] += 1
        co_occurrence[word2][word1] += 1
    # 단어 쌍이 함께 등장할 때마다 해당 쌍의 빈도수를 1씩 증가

# 동시 출현 매트릭스를 DataFrame으로 변환
co_occurrence_df = pd.DataFrame(co_occurrence)
# 동시 출현 빈도를 데이터프레임으로 변환하여 표 형태로 정리

# 결과를 CSV 파일로 저장
co_occurrence_df.to_csv("1_mode_matrix.csv",encoding='cp949')
# 결과를 CSV 파일로 저장하여 나중에 분석할 수 있도록 함

co_occurrence_df.head()  # 결과의 첫 부분만 보여줌
# 결과의 첫 몇 줄을 화면에 출력하여 확인


,지능,인공,날씨,기술,서울,지방,개발,인천,교육,기온,...,오늘,산업,대전,아침,시간,세계,활용,제주,분야,강릉
지능,0,97,0,30,10,0,27,1,14,0,...,0,15,3,1,4,7,23,0,15,0
인공,97,0,0,30,10,0,27,1,13,0,...,0,15,3,1,4,7,23,0,14,0
날씨,0,0,0,0,42,22,0,42,0,42,...,42,0,39,21,24,0,0,34,0,39
기술,30,30,0,0,7,0,14,0,8,0,...,0,9,2,0,3,7,15,0,10,0
서울,10,10,42,7,0,22,4,42,4,42,...,42,4,40,21,27,2,3,34,4,39


In [18]:
# 선택단어 기준
# from itertools import combinations
# data = data.astype('str')
# chars_to_remove = r"[\[\]',]"
# data = data.replace(chars_to_remove, '', regex=True)
# my_selected_words = ['단어1', '단어2', '단어3', ...]  # 내가 선택한 단어 리스트
# # 동시 출현 빈도 계산을 위한 빈 딕셔너리 생성
# co_occurrence = {word: {other_word: 0 for other_word in my_selected_words} for word in my_selected_words}
# # 각 리뷰에 대해 동시 출현 빈도 계산
# for review in data[column_name]:
#     # 리뷰에서 내가 선택한 단어 찾기
#     words_in_review = set(word for word in review.split() if word in my_selected_words)
#     # 단어 쌍에 대해 동시 출현 빈도 증가
#     for word1, word2 in combinations(words_in_review, 2):
#         co_occurrence[word1][word2] += 1
#         co_occurrence[word2][word1] += 1
# # 동시 출현 매트릭스를 DataFrame으로 변환
# co_occurrence_df_seleteced = pd.DataFrame(co_occurrence)
# co_occurrence_df_seleteced.to_csv(f"{filename}_1_mode_matrix_selected.csv",encoding='cp949')
# co_occurrence_df_seleteced.head()  # 결과의 첫 부분만 보여줌